<a href="https://colab.research.google.com/github/inepicone/product_footprint/blob/main/mvp_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗺️ **Eco-Impact Calculator 🌍**

This notebook estimates the environmental impact of transporting a product based on its origin and your location.

🔹 **How it works:**
1. The script randomly selects a product from the **Open Food Facts** database via its API.
2. It geolocates both the user and the product's origin using **geopy**.
3. It calculates the straight-line distance between both points.
4. It estimates the fuel consumption (assuming 3 km per liter of fuel).
5. Finally, it plots the route on an interactive map.

📌 **Note:**  
- The distance is shown as a straight **LINE**, not a real-world route.  
- The map will be displayed directly within the notebook. 🚀


In [5]:
# mvp_v2 - Carbon footprint calculation using coordinates and map visualization

# Install necessary libraries (ensure they are installed correctly)
!pip install geopandas folium geopy requests

In [31]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.distance import geodesic
import geopandas as gpd
import folium
import requests
import random
from time import sleep
from IPython.display import display


In [34]:
# Function to fetch a random product from Open Food Facts
def get_product():
    url = "https://world.openfoodfacts.org/cgi/search.pl?search_simple=1&action=process&json=1&page_size=50"
    response = requests.get(url)
    data = response.json()

    valid_products = []

    if "products" in data and len(data["products"]) > 0:
        for product in data["products"]:
            name = product.get("product_name", "Unknown")
            origin_country = product.get("origins", "").strip()

            if origin_country and origin_country.lower() != "unknown":
                valid_products.append((name, origin_country))

    # checking if we found a valid product
    if not valid_products:
        print("No valid products found with origin country.")
        return None, None

    # choosing one random
    selected_product = random.choice(valid_products)

    print(f"Selected product: {selected_product[0]} from {selected_product[1]}")  # Debug

    return selected_product

# Function to get coordinates of a city
def get_coordinates(location):
    geolocator = Nominatim(user_agent="my-geocoding-app")

    try:
        location = geolocator.geocode(location)
        if location:
            return location.latitude, location.longitude
        else:
            return None
    except GeocoderTimedOut:
        print(f"Geocoding service timed out for {location}. Retrying...")
        sleep(2)  # 2 seg waiting to reboot
        return get_coordinates(location)

# Function to calculate distance and fuel consumption
def calculate_impact(user_location, product_origin_location):
    user_coords = get_coordinates(user_location)
    product_coords = get_coordinates(product_origin_location)

    if user_coords and product_coords:
        distance_km = geodesic(user_coords, product_coords).kilometers
        fuel_consumption_liters = distance_km / 3  # we asumed 3km per litre of fuel
        return distance_km, fuel_consumption_liters  # Only 2
    else:
        return None, None

In [36]:
# Simulate getting a random product and calculate impact

# Get random product
product, origin_country = get_product()

# User input
user_location = input("Enter your location (e.g., Madrid, Spain): ")

user_coords = get_coordinates(user_location)
product_coords = get_coordinates(origin_country)

if user_coords and product_coords:
    # Creating the map
    map_center = [(user_coords[0] + product_coords[0]) / 2,
                  (user_coords[1] + product_coords[1]) / 2]

    m = folium.Map(location=map_center, zoom_start=5)

    # adding marks
    folium.Marker(user_coords, popup='User Location', icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker(product_coords, popup=f'Product Origin: {origin_country}', icon=folium.Icon(color="green")).add_to(m)

    # line between cities
    folium.PolyLine([user_coords, product_coords], color="red", weight=2.5, opacity=1).add_to(m)

    # showing the map.
    display(m)
else:
    print("Could not calculate distance.")


Selected product: Cosumar Sucre En Lingot Pour Le Thé 1 kg from Maroc,Morocco
Enter your location (e.g., Madrid, Spain): Tokio, Japan
